# Step 1: Simulation

## a) Run the Monte Carlo simulation
The simulations are called from a standalone python file, as they take quite long.
It is recommended to run them in a `screen` with

```sh
nice python 01-simulate.py
```

Note: Supervise running simulations with 
```sh
while true; do clear; tail -n1 output/*/*.simu.log | grep '>>>'; sleep 10; done
```

## b) Check for successful execution

In [1]:
distances = [15, 35]
doubleplanes = [8, 12, 20, 30]
energies = [200, 600, 1000]
erels = [100, 500, 1000, 2000, 3000]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["bert", "bic", "inclxx"]

In [2]:
import pandas as pd
import numpy as np
import joblib
import os
import ROOT
from helpers import filename_for

ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


In [3]:
def processed_events(distance, doubleplane, energy, erel, neutron, physics):
    filename = filename_for(distance, doubleplane, energy, erel, neutron, physics, ".simu.root")
    if os.path.isfile(filename):
        try:
            tfile = ROOT.TFile.Open(filename)
            ttree = tfile.Get("evt")
            num_events = int(ttree.GetEntries())
            return (filename, num_events)
        except:
            pass
    return (filename, 0)

In [4]:
evts = pd.DataFrame(
    joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(
        joblib.delayed(processed_events)(
            distance=distance, doubleplane=doubleplane, energy=energy, erel=erel, neutron=neutron, physics=physics,
        )
        for distance in distances
        for energy in energies
        for doubleplane in doubleplanes
        for neutron in neutrons
        for erel in erels
        for physics in physicss
    )
)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 1138 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 1688 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed:   21.1s finished


In [5]:
pd.options.display.max_rows = 5
evts.columns = ["File", "Events"]
evts.sort_values(by=["Events", "File"], ascending=[True, True], inplace=True)
evts.style.hide_index().format({"Events": "{:d}"})
display(evts)

,File,Events
816,output/bert/15m_12dp_1000AMeV_1000keV_1n.simu....,10000
831,output/bert/15m_12dp_1000AMeV_1000keV_2n.simu....,10000
...,...,...
1505,output/inclxx/35m_8dp_600AMeV_500keV_5n.simu.root,10000
1520,output/inclxx/35m_8dp_600AMeV_500keV_6n.simu.root,10000
